In [15]:
import pandas as pd
import time
import os
from anthropic import Anthropic
client = Anthropic(
    api_key=os.environ.get("ANTHROPIC_API_KEY"), # make sure you create your .env file first
)

In [3]:
sample_ojo_data = pd.read_parquet("s3://open-jobs-indicators/hackathon_data/hackathon_data.parquet")

In [27]:
print(f"Loaded {len(sample_ojo_data)} rows")
sample_ojo_data.head()

Loaded 508907 rows


,id,job_title_raw,created,description,itl_3_code,itl_3_name,min_annualised_salary,max_annualised_salary,anthropic_response
0,42442182,Warehouse Operative,2021-04-11,[ Logistic People are recruiting now in Corby ...,TLF25,North Northamptonshire,18895.5,21274.5,Warehousing skills</skill>
1,46026103,Account Executive - Large Format Print/POS,2022-03-07,[ Our client is a very well-established name i...,TLE44,Calderdale and Kirklees,25000.0,30000.0,Account management</skill>
2,48029597,Front End Developer,2022-08-31,[ Front End Developer - Contract - Up to £450 ...,TLI,London,104000.0,117000.0,React</skill>
3,42233243,HGV 1 Driver,2021-03-12,[ HGV 1 Driver We are recruiting for a HGV 1 D...,TLJ11,Berkshire,27300.0,29250.0,Driving a Heavy Goods Vehicle (HGV) Category C...
4,47189444,Commercial Insurance SME Account Handler,2022-06-15,[Our client a very well established and busy i...,TLJ14,Oxfordshire,25000.0,27000.0,Commercial insurance knowledge and experience<...


In [28]:
def prompt(row):
    return f"""
Please pick out the main skill required from this job description:

{row['job_title_raw']}
{row['description']}

Answer directly, inside <skill> tags.
""".strip()

In [26]:

t0 = time.time()
for index, row in sample_ojo_data.iterrows():
    if (index > 10):
        break
    
    message = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=1024,
        messages=[
            {
                "role": "user",
                "content": prompt(row),
            },
            {
                "role": "assistant",
                "content": "<skill>",
            }
        ],
    )
    response = message.content[0].text
    print(f"===\n\n{row['job_title_raw']}\n{row['description']}\n\n<skill>{response}")

    # save the response to a new column in the dataframe
    sample_ojo_data.loc[index, "anthropic_response"] = response
t1 = time.time()
print(f"Took {((t1-t0) / (index+1)):.2f} seconds per row")

===

Warehouse Operative
[ Logistic People are recruiting now in Corby for Warehouse Operatives!!  Available shifts : Mon-Fri Rotating 0600-1400/1400-2200 Various shifts available Warehouse Details: Ambient Warehouse, Multiclient Site Responsibilities: Picking and packing, unloading and loading pallets within the warehouse, rework Rate: Day shift rate: £9.69ph (£14.54ph for overtime paid over 37.5h) Night shift rate: £10.91ph (16.03ph for overtime paid over 37.5h) Requirements:  Basic understanding of English (speaking and writing) is mandatoryPrevious experience in a warehouse is desirable but not essential as training is provided. Benefits: Fantastic rate of £9.69ph and £14.54 for Overtime – paid weekly! Recommend a friend bonuses – no limits on number of recommended friends! 24-hour support from the Logistics People team Call our office on   ]

<skill>Warehousing skills</skill>
===

Account Executive - Large Format Print/POS
[ Our client is a very well-established name in large form